In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
%load_ext autoreload
%autoreload 2

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [ ]:
#disfunctional
pretrained_weights = torch.load("../features/VGG16_relu5_3.pickle")

In [4]:
def load_dataset():
    data_path = "../data/imgs_small"
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=32,
        num_workers=0,
        shuffle=True
    )
    return train_loader



In [6]:
train_data = load_dataset()

In [15]:
# print (train_data.__getitem__(0).shape)
train_iter = iter(train_data)

a, b = train_iter.next()
print (a.shape)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 511 and 512 in dimension 2 at c:\programdata\miniconda3\conda-bld\pytorch_1533090623466\work\aten\src\th\generic/THTensorMath.cpp:3616

In [3]:
vgg16 = models.vgg16_bn()
vgg16.load_state_dict(pretrained_weights)
print(vgg16.classifier[6].out_features) # 1000 2

NameError: name 'pretrained_weights' is not defined